<div> <center><img src="https://storage.googleapis.com/open-ff-common/openFF_logo.png" width="100"/></div>
<h1><center>Proprietary notebook: version Oct 2023</center></h1>

In [ ]:
!git clone https://github.com/gwallison/intg_support.git &>/dev/null;
!pip install itables  &>/dev/null;
!pip install geopandas  &>/dev/null;

import urllib
urllib.request.urlretrieve('https://storage.googleapis.com/gwa-test/georef-united-states-of-america-county.geojson',
                            'counties.geojson')

In [ ]:
%run intg_support/proprietary_aug_2023.py

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import gca, mpl

%matplotlib inline
import seaborn as sns

# get full set of data
import intg_support.geo_tools as gt
work_dir = ''
# work_dir = './tmp'
get_fulldf(work_dir=work_dir) # this downloads most recent from the web.
df = get_df(os.path.join(work_dir,'full_df.parquet'))
df = df[df.in_std_filtered]

# NOW HAS FILTER to remove disclosures without chemicals!!
df = df[df.ingKeyPresent]

# uncomment following line to limit the dates
df = df[(df.date.dt.year>2013)&(df.date.dt.year<2023)]


In [ ]:
# make a proprietary only data set
prop_df = df[df.bgCAS=='proprietary'].copy()


# identify as many proprietary proppants as we can

In [ ]:
s = """30/70 Permian
aa-400 (aluminum oxide)
aluminum oxide
amorphous Silica
amorphous silica
amorphous silicia
ceramic microspheres
ceramic microspheres/glutaraldehyde
ceramic propant
ceramic proppant
ceramic proppant proprietary
copolymer resin fracturing proppant
corundum
crys4808-60-7talline sio2
crystalline cristobalite
crystalline silica
crystalline silica (quartz)
crystalline silica (quartz), proprietary
crystalline silica, quartz
crystalline silica: cristobalite
crystalline silica: quartz (sio2)
crystalline silica(quartz),proprietary
crystalline sio2
fumed silica
hydrophobic silica
hydrated magnesium silicate
magnesium silicate hydrate (talc)
meghemite
non- crystalline silica (impurity)
non-crystalline silica 
proppant
proprietary quartz
proprietary silica 
quartz
quartz (sio2)
sand
silica substrate
silica substitute with bonded coatings
silicate minerals - ts
zinc oxide"""
propp_lst = s.split('\n')

prop_df['is_proppant'] = prop_df.IngredientName.isin(propp_lst)
print(f'Total number of identified proprietary proppant records: {prop_df.is_proppant.sum()}')
prop_df[prop_df.is_proppant].IngredientName.value_counts()

In [ ]:
# remove one obvious error point
import numpy as np
prop_df.calcMass = np.where(prop_df.calcMass>100000000,np.NaN,prop_df.calcMass)

In [ ]:
from pylab import gca, mpl
ax = prop_df[prop_df.is_proppant].plot('date','calcMass',style='o',alpha=.5,legend=False)
ax.set_title('Mass of proprietary records with proppant ingredient name')
ax.set_xlabel('Year')
ax.set_ylabel('Mass (lbs.) of individual proprietary records')
# prop_df[~prop_df.is_proppant].plot('date','calcMass', style='o',alpha=.2, ax=ax, legend=False)
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

In [ ]:
import seaborn as sns
import numpy as np

def my_formatter(x, pos):
     return "{}".format('{x:,.4f}' if x<1 else '{x:,.0f}' )
    
sns.set_theme(style="ticks")
prop_df['logMass'] = np.where(prop_df.calcMass>0.00001,np.log10(prop_df.calcMass),np.NaN) 
prop_df['year'] = prop_df.date.dt.year
# ax = sns.boxplot(data=prop_df[~prop_df.is_proppant], x='year',y='logMass',showfliers=False)
ax = sns.boxplot(data=prop_df[(~prop_df.is_proppant)&(prop_df.calcMass>.001)], x='year',y='calcMass',
                 showfliers=False, color='skyblue')
# sns.regplot(data=prop_df[(~prop_df.is_proppant)&(prop_df.calcMass>.001)], x='year',y='calcMass',
#             ax=ax, scatter=False)
# # ax.set_title('Mass of non-proppant proprietary records (log(Mass))')
ax.set_ylabel('log of calculated mass')
plt.yscale("log")
ax.tick_params(labelright=True, right=True, which='both')
# ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.4f}'))
# ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter())

In [ ]:
prop_df[~prop_df.is_proppant].groupby('year')['calcMass'].agg(['median','mean','sum','count'])

# basic proprietary stats (added 8/28)

In [ ]:
print(f'Total calculated mass of proprietary records: {round_sig(prop_df.calcMass.sum(),3)} lbs.')
prop_df['added_year'] = prop_df.date_added.dt.year
print(f'Calculated mass of proprietary records added in 2023: {round_sig(prop_df[prop_df.added_year==2023].calcMass.sum(),3)} lbs.')

prop_df['year'] = prop_df.date.dt.year
# with pd.option_context("display.float_format", "{:,.0f}".format):
gb = prop_df.groupby('year',as_index=False)['calcMass'].sum()
gb['rounded'] = gb.calcMass.map(lambda x: round_sig(x,3))
gb

In [ ]:
total_num_disc = df.UploadKey.unique().size
num_disc_with_prop = prop_df.UploadKey.unique().size
print(f'Total num of disclosures in set:      {total_num_disc:,}')
print(f'Total with at least one propriertary: {num_disc_with_prop:,}')
print(f'Overall Percent with proprietary:     {num_disc_with_prop/total_num_disc:.2%}')

# show all the Purpose fields
You could also look at the [Browser's version](https://storage.googleapis.com/open-ff-browser/proprietary/analysis_proprietary.html#raw) of this.  Extra long purposes (using indicating multiple products) are truncated and combined.  They are not very useful...

In [ ]:
prop_df.groupby('Purpose',as_index=False).size()

# make graphs of operators and suppliers

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# cas = prop_df.bgCAS.iloc[0]
# use the most common name given in FF for the label
gb1 = prop_df.groupby('bgOperatorName')['OperatorName'].agg(lambda x: x.value_counts().index[0])
gb1 = gb1.reset_index()
gb1.columns = ['bgOperatorName','op_common']
mg = pd.merge(prop_df,gb1,on='bgOperatorName',how='left')
newmg = mg.groupby('op_common')['UploadKey'].count().sort_values(ascending=False)
ax = newmg[:15].plot.barh(figsize=(7,7))
ax.tick_params(axis="y", labelsize=14)
ax.tick_params(axis="x", labelsize=14)
plt.xlabel('Number of records',fontsize=16);
plt.ylabel('Operating Company',fontsize=16);
plt.title(f'Number of "proprietary" records, by operator',fontsize=16);

In [ ]:
gb_full = df.groupby(['bgOperatorName','UploadKey'],as_index=False).size()
gb_full = gb_full.groupby('bgOperatorName')['UploadKey'].count().reset_index()
gb_full.columns = ['bgOperatorName','disclosure_cnt_all']

gb_prop = prop_df.groupby(['bgOperatorName','UploadKey'],as_index=False).size()
gb_prop = gb_prop.groupby('bgOperatorName')['UploadKey'].count().reset_index()
gb_prop.columns = ['bgOperatorName','disclosure_cnt_prop']



In [ ]:
mg = pd.merge(gb_full,gb_prop,on='bgOperatorName',how='left')
mg = pd.merge(mg,gb1,on='bgOperatorName',how='left')
mg = pd.merge(mg,newmg.reset_index(),on='op_common',how='left')
mg = mg.rename({'UploadKey':'num_prop_records'},axis=1)
mg = mg[mg.disclosure_cnt_prop>0]
mg.disclosure_cnt_prop.fillna(0,inplace=True)
mg['prop_perc'] = mg.disclosure_cnt_prop/mg.disclosure_cnt_all * 100
mg.sort_values('disclosure_cnt_all',ascending=False)

# Suppliers

Here we need to remove records that are systems approach

In [ ]:
c = df.CASNumber.str.lower().str.contains('listed below')
c1 = df.IngredientName.str.lower().str.contains('listed below')
print(f'Number of disclosures that are systems approach removed from consideration : {df[c1|c].UploadKey.unique().size}')
upk = df[c1|c].UploadKey.unique().tolist()
prop_df_sup = prop_df[~prop_df.UploadKey.isin(upk)]
print(f'Number of disclosures in the supplier analysis: {prop_df_sup.UploadKey.unique().size}')

In [ ]:
# records to exclude
not_comp = ['MISSING','Listed Above']

gb1 = prop_df_sup.groupby('bgSupplier')['Supplier'].agg(lambda x: x.value_counts().index[0])
gb1 = gb1.reset_index()
gb1.columns = ['bgSupplier','sup_common']
mg = pd.merge(prop_df_sup,gb1,on='bgSupplier',how='left')

mg = mg[~mg.sup_common.isin(not_comp)]

ax = mg.groupby('sup_common')['UploadKey'].count()\
         .sort_values(ascending=False)[:15].plot.barh(figsize=(7,7))
ax.tick_params(axis="y", labelsize=14)
ax.tick_params(axis="x", labelsize=14)
plt.xlabel('Number of records',fontsize=16);
plt.ylabel('Supplier Company',fontsize=16);
plt.title(f'Number of records declared proprietary, by supplier',fontsize=16);

In [ ]:
gb_full = df.groupby(['bgSupplier'],as_index=False).size()
gb_full.columns = ['bgSupplier','record_cnt_all']

gb_prop = prop_df_sup.groupby(['bgSupplier'],as_index=False).size()
gb_prop.columns = ['bgSupplier','record_cnt_prop']

# gb_precs = prop_df.groupby

In [ ]:
mg = pd.merge(gb_full,gb_prop,on='bgSupplier',how='left')
mg = pd.merge(mg,gb1,on='bgSupplier',how='left')
mg.record_cnt_prop.fillna(0,inplace=True)
mg['prop_perc'] = mg.record_cnt_prop/mg.record_cnt_all * 100
mg.sort_values('record_cnt_all',ascending=False)

# MAPS

In [ ]:
import geopandas as gpd
import folium
import numpy as np

import branca.colormap as cm
linear = cm.LinearColormap(['green','yellow','red'], vmin=3., vmax=10.)
linear

def fix_county_names(df):
    trans = {'mckenzie':'mc kenzie',
             'dewitt':'de witt',
             'mcclain':'mc clain',
             'mcintosh':'mc intosh',
             'mckean':'mc kean',
             'mcmullen':'mc mullen'}
    for wrong in trans.keys():
        df.CountyName = np.where(df.CountyName==wrong,trans[wrong],df.CountyName)
    return df

def create_county_choropleth(data,
                             start_loc=[40, -96],start_zoom = 6,
                             custom_scale = [], plotlog = True,
                             legend_name = 'Test legend',
                             show_only_data_states=True,
                             fill_color = 'YlOrRd',
                             #popup_enabled=True, tooltip_enabled=False,
                             fields = ['CountyName','orig_value'],
                             aliases = ['County: ','data: ']):
    fn = r"counties.geojson"
    if len(data)<1:
        print('No mappable data')
        return
    geojson = gpd.read_file(fn)
    data['orig_value'] = data.value

    geojson['StateName'] = geojson.ste_name.str.lower()
    geojson['CountyName'] = geojson.coty_name.str.lower()
    geojson = fix_county_names(geojson)
    working = geojson[['StateName','CountyName','coty_code','geometry']]
    #geojson = geojson.to_crs(5070)
    working = pd.merge(working,data,on=['StateName','CountyName'],how='left')
    #print(geojson.info())
    if start_loc==[]:
        start_loc = [geojson.geometry.centroid.x.mean(),geojson.geometry.centroid.y.mean()]
    f = folium.Figure(width=600, height=400)
    m = folium.Map(location= start_loc, tiles="openstreetmap",
                   zoom_start=start_zoom).add_to(f)
    if plotlog:
        working.value = np.log10(working.value+1)
        legend_name = legend_name + ' (log transformed)'
    working.orig_value.fillna('no data',inplace=True)
    
    if custom_scale==[]:
        custom_scale = (working['value'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
    if show_only_data_states:
        gb = data.groupby(['StateName','CountyName'],as_index=False)['value'].first()
        datalst = []
        for i,row in gb.iterrows():
            datalst.append((row.StateName,row.CountyName))
        wlst = []
        working['tup'] = list(zip(working.StateName.tolist(),working.CountyName.tolist()))
        geojson['tup'] = list(zip(geojson.StateName.tolist(),geojson.CountyName.tolist()))
        
#         working = working[working.StateName.isin(data.StateName.unique().tolist())]
#         geojson = geojson[geojson.StateName.isin(data.StateName.unique().tolist())]
#         c1 = working.CountyName.isin(data.CountyName.unique().tolist())
#         c2 = working.StateName.isin(data.StateName.unique().tolist())
#         c3 = geojson.CountyName.isin(data.CountyName.unique().tolist())
#         c4 = geojson.StateName.isin(data.StateName.unique().tolist())
        working = working[working.tup.isin(datalst)]
        geojson = geojson[geojson.tup.isin(datalst)]
    working.StateName = working.StateName.str.title()
    working.CountyName = working.CountyName.str.title()
    #print(f'States in geojson: {working.StateName.unique().tolist()}')

    linear = cm.LinearColormap(['green','yellow','red'], vmin=3., vmax=10.)
    linear
    
    folium.Choropleth(
                geo_data=geojson,
                data=working,
                columns=['coty_code', 'value'],  #Here we tell folium to get the fips and plot values for each state
                key_on='feature.properties.coty_code',
                threshold_scale=custom_scale, #use the custom scale we created for legend
                #fill_color='YlOrRd',
                fill_color=fill_color,
                nan_fill_color="gainsboro", #Use white color if there is no data available for the area
                fill_opacity=0.7,
                line_opacity=0.4,
                line_weight=0.4,
                legend_name= legend_name, #title of the legend
                highlight=True,
                line_color='black').add_to(m) 
    
    folium.features.GeoJson(
                data=working,
                name='',
                smooth_factor=2,
                style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                popup=folium.features.GeoJsonPopup(
                    fields=fields,
                    aliases=aliases, 
                    localize=True,
                    sticky=False,
                    labels=True,
                    style="""
                        background-color: #F0EFEF;
                        border: 2px solid black;
                        border-radius: 3px;
                        box-shadow: 3px;
                    """,
                    max_width=800,),
                        highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                    ).add_to(m)  

    

# fit_bounds needs work: https://stackoverflow.com/questions/58162200/pre-determine-optimal-level-of-zoom-in-folium
#     sw = data[['bgLatitude', 'bgLongitude']].min().values.tolist()
#     ne = data[['bgLatitude', 'bgLongitude']].max().values.tolist()

#     m.fit_bounds([sw, ne]) 
    display(f)


def CountyMap(df):
    state_list = df.bgStateName.unique().tolist()
    #start_loc = get_geog_center(state_list)
    #print(statename,start_loc)
    cond = (df.loc_within_state=='YES')&(df.loc_within_county=='YES')
    if cond.sum()==0:  # no valid fracks for this state
        display(md('## No mappable fracks for this operator!'))
        # display(md(f'Any data in this state set may be labeled incorrectly as {statename}'))
        return
    gb = df[cond].groupby(['bgStateName','bgCountyName',
                                                   'UploadKey'],as_index=False)['bgCAS'].count()
    gb = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['UploadKey'].count().rename({'bgStateName':'StateName',
                                                                                                'bgCountyName':'CountyName',
                                                                                                'UploadKey':'value'},
                                                                                                axis=1)    
    zoom = 3.6
    create_county_choropleth(gb,plotlog=True,#plotlog=True,
                             custom_scale= [0,1,2,3,4],
                             #start_loc=start_loc, # center of state's data
                             legend_name='Number of FracFocus disclosures',
                             start_zoom=zoom,fields=['StateName','CountyName','orig_value'],
                             aliases=['State: ','County: ','Number Fracking disclosures: '])

## Map by number of disclosures
Does this color scheme support our points?

In [ ]:
CountyMap(df)

# make map with proprietary fraction

In [ ]:
# first make data frame with number of fracks by county
gb_all = df.groupby(['bgStateName','bgCountyName','UploadKey'],as_index=False).size()
gb_all = gb_all.groupby(['bgStateName','bgCountyName'],as_index=False)['UploadKey'].count()                    
# gb_all

# now make data frame with number of fracks with at least one prop. chem by county
gb_prop = prop_df.groupby(['bgStateName','bgCountyName','UploadKey'],as_index=False).size()
gb_prop = gb_prop.groupby(['bgStateName','bgCountyName'],as_index=False)['UploadKey'].count()                    
gb_prop.columns = ['bgStateName','bgCountyName','UploadKey_prop']
gb_prop

mg = pd.merge(gb_all,gb_prop,on=['bgStateName','bgCountyName'],how='left')
mg.UploadKey_prop.fillna(0,inplace=True)
mg['perc_prop'] = mg.UploadKey_prop/mg.UploadKey *100
#mg[mg.UploadKey>0].perc_prop.hist(bins=5)

mg['prop_dev'] = mg.perc_prop - 82

mg.UploadKey_prop.sum()/mg.UploadKey.sum() *100

## %disclosures_with_proprietary: 4 simple categories

In [ ]:
def CountyPropMap(df):
    state_list = df.bgStateName.unique().tolist()
    zoom = 3.6
    df['value'] = df.perc_prop
    # df['value'] = df.prop_dev
    df['StateName'] = df.bgStateName
    df['CountyName'] = df.bgCountyName
    create_county_choropleth(df,plotlog=False,
                             custom_scale=[0,25,50,75,100],
                             fill_color='RdBu_r',
                             #start_loc=start_loc, # center of state's data
                             legend_name='Percent of disclosures with at least one proprietary record',
                             start_zoom=zoom,fields=['bgStateName','bgCountyName','orig_value','UploadKey'],
                             aliases=['State: ','County: ','% disc with proprietary: ','total num of disclosures'])
    
CountyPropMap(mg[mg.UploadKey>5].copy()) # county must have more than 5 FF disclosures

# Compare to Trickey 2020
state-wide proprietary rates

In [ ]:
gb_all = df.groupby(['bgStateName','UploadKey'],as_index=False).size()
gb_all = gb_all.groupby('bgStateName',as_index=False)['UploadKey'].count()
gb_all.columns = ['bgStateName','num_all_disc']

gb_prop = df[df.bgCAS=='proprietary'].groupby(['bgStateName','UploadKey'],as_index=False).size()
gb_prop = gb_prop.groupby('bgStateName',as_index=False)['UploadKey'].count()
gb_prop.columns = ['bgStateName','num_prop_disc']

mg = pd.merge(gb_all,gb_prop,on='bgStateName',how='left')
mg.num_prop_disc.fillna(0,inplace=True)

mg['state_prop_percent'] = mg.num_prop_disc/mg.num_all_disc *100
mg.sort_values('num_all_disc',ascending=False)

# ambiguousID

In [ ]:
amb_df = df[df.bgCAS=='ambiguousID'].copy()
amb_df['has_mass'] = amb_df.calcMass>0
amb_df.has_mass.value_counts()

## what are the ambiguousID `IngredientName`s


In [ ]:
amb_df[amb_df.has_mass].IngredientName.value_counts()

## what are the ambiguousID `IngredientName`s  (Just the big ones)
Mostly water-type things.  A handful of proppants

In [ ]:
amb_df[amb_df.calcMass>100000].IngredientName.value_counts()

# AmbiguousID Proppants: Number and mass

In [ ]:
proppants = ['silica substrate',
 'remainder is made up of various other oxides and trace elements, of which cao, mgo, and fe2o3 are the largest percentages',
 'cyrstalline silica','mix of various oxides (cao, mgo, and fe2o4','resin coated fracturing proppant',
 '40/70 ppc','mix of various oxides (cao, mgo, and fe2o3',
 'mix of various oxides (cao, mgo, and fe203','mix of various oxides (cao, mgo, and fe204',
 'crystalline silica, quartz','100 mesh sand','ceramic',
 '20/40 pc','crystalline silica,quartz','aluminum oxide','nfidb:sand-200 mesh silica',
 'nfidb:200 mesh ssa-1','non-crystalline silica (impurity)',
 'silica in form of quartz','40/70 white','mulite']
c = amb_df.IngredientName.isin(proppants)

print(f'Number of ambiguousID proppants = {len(amb_df[c])}; calculated mass= {round(amb_df[c].calcMass.sum(),0):,} lbs.')

## Likely water records in ambiguousID

In [ ]:
c = amb_df.IngredientName.str.contains('water')

print(f'Number of ambiguousID **water** = {len(amb_df[c])}; calculated mass= {round(amb_df[c].calcMass.sum(),0):,} lbs.')

# Proprietary plot (added back 8/28)

In [ ]:

from matplotlib.offsetbox import AnchoredText
def proprietary_plot(df,plot_title='TEST',minyr=2014,maxyr=2022):
    # df = df.copy()
    df['year'] = df.date.dt.year
    df = df[df.year<=maxyr]
    df = df[df.year>=minyr]
    prop = df.bgCAS=='proprietary'
    gb = df[prop].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numprop'},axis=1)
    gb1 = df[df.is_valid_cas].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numvalid'},axis=1)
    gb2 = df.groupby('UploadKey',as_index=False)['date'].first()
    mg = pd.merge(gb2,gb,on='UploadKey',how='left')
    mg = pd.merge(mg,gb1,on='UploadKey',how='left')
    mg.fillna(0,inplace=True) # there will be disclosures with 0 proprietary; need to fill
    mg['percProp'] = (mg.numprop / mg.numvalid) * 100

    mg['propCut'] = pd.cut(mg.percProp,right=False,bins=[0,0.0001,10,25,50,101],
                          labels=['no proprietary claims','up to 10% proprietary claims',
                                  'between 10 and 25% proprietary claims',
                                  'between 25 and 50% proprietary claims',
                                  'greater than 50% proprietary claims'])
    mg['year'] = mg.date.dt.year
    # tmp = mg.groupby(['year','propCut'],as_index=False).size()
    # print(tmp.pivot(index='year',columns='propCut',values='size'))
    
    out = mg.drop(['date','UploadKey'],axis=1)
    t = out[out.numvalid>0].groupby(['year','propCut'],as_index=False)['numvalid'].count()
    sums = t.groupby('year',as_index=False)['numvalid'].sum().rename({'numvalid':'tot'},axis=1)
    t = pd.merge(t,sums,on='year',how='left')
    t['PercentProp'] = t.numvalid/t.tot *100

    piv = t.pivot(index='year', columns='propCut', values='PercentProp')
    
    with pd.option_context("display.float_format", "{:,.1f} %".format):
        iShow(piv)

    ax = piv.plot.area(figsize=(12,7),ylim=(0,100),xlim=(minyr,maxyr),colormap='Reds')
    ax.set_title(f'Percentage of valid records that are Trade Secret claims at the disclosure level', fontsize=16)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1], title='Disclosure Proprietary\nPercentage class\n',
              loc='upper left',bbox_to_anchor=(1, 1))
    ax.set_ylabel('Percentage of disclosures', fontsize=16)
    ax.set_xlabel('Year', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.suptitle(f'{plot_title}',fontsize=24)

    gb = df.groupby(['year','UploadKey'],as_index=False)['bgCAS'].count()
    gb = gb.groupby('year',as_index=False)['UploadKey'].count()#.rename({'UploadKey':'number of disclosures'},axis=1)
    s = 'Number of disclosures by year:\n\n'
    for i,row in gb.iterrows():
        s+= f'   {row.year}: {row.UploadKey:7,} \n'
    at2 = AnchoredText(s,
                       loc='lower left', prop=dict(size=10), frameon=False,
                       bbox_to_anchor=(1., 0.),
                       bbox_transform=ax.transAxes
                       )
    at2.patch.set_boxstyle("square,pad=0.")
    ax.add_artist(at2)

    
# test = 'pennsylvania'
# variable = 'bgStateName'
testtitle = 'Trade Secret frequency'
proprietary_plot(df,testtitle,minyr=2014,maxyr=2022)


###  Break up the categories for plotting and analysis

In [ ]:
import scipy.stats as stats
def new_plot(df, category='no proprietary designations'):
    df['year'] = df.date.dt.year
    prop = df.bgCAS=='proprietary'
    gb = df[prop].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numprop'},axis=1)
    gb1 = df[df.is_valid_cas].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numvalid'},axis=1)
    gb2 = df.groupby('UploadKey',as_index=False)['date'].first()
    mg = pd.merge(gb2,gb,on='UploadKey',how='left')
    mg = pd.merge(mg,gb1,on='UploadKey',how='left')
    mg.fillna(0,inplace=True) # there will be disclosures with 0 proprietary; need to fill
    mg['percProp'] = (mg.numprop / mg.numvalid) * 100

    mg['propCut'] = pd.cut(mg.percProp,right=False,bins=[0,0.0001,10,25,50,101],
                          labels=['no proprietary designations','up to 10% proprietary designations',
                                  'between 10 and 25% proprietary designations',
                                  'between 25 and 50% proprietary designations',
                                  'greater than 50% proprietary designations'])
    mg['year'] = mg.date.dt.year
    
    out = mg.drop(['date','UploadKey'],axis=1)
    t = out[out.numvalid>0].groupby(['year','propCut'],as_index=False)['numvalid'].count()
    sums = t.groupby('year',as_index=False)['numvalid'].sum().rename({'numvalid':'tot'},axis=1)
    t = pd.merge(t,sums,on='year',how='left')
    t['PercentProp'] = t.numvalid/t.tot *100

    g = sns.FacetGrid(t,col='propCut',col_wrap=2,height=4)
    g.map(sns.regplot,'year','PercentProp')
    for cat in mg.propCut.unique().tolist()[:-1]:
        print(f'Regression stats for <{cat}>')
        subdf = t[t.propCut==cat]
        print(stats.linregress(subdf.year,subdf.PercentProp),'\n')
new_plot(df,'no proprietary designations')

### Try analysis of just mean proportion over years

In [ ]:
def single_plot(df):
    df['year'] = df.date.dt.year
    prop = df.bgCAS=='proprietary'
    gb = df[prop].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numprop'},axis=1)
    gb1 = df[df.is_valid_cas].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numvalid'},axis=1)
    gb2 = df.groupby('UploadKey',as_index=False)['date'].first()
    mg = pd.merge(gb2,gb,on='UploadKey',how='left')
    mg = pd.merge(mg,gb1,on='UploadKey',how='left')
    mg.fillna(0,inplace=True) # there will be disclosures with 0 proprietary; need to fill
    mg['percProp'] = (mg.numprop / mg.numvalid) * 100

    mg['year'] = mg.date.dt.year
    ygb = mg.groupby('year',as_index=False)['percProp'].mean()

    print(f'Regression stats')
    print(stats.linregress(ygb.year,ygb.percProp),'\n')
    sns.regplot(data=ygb,x='year',y='percProp',scatter=True)
single_plot(df)

## Oct 2023.  Median water volume

In [ ]:
from math import log10, floor
def round_sig(x, sig=2,guarantee_str=''):
    try:
        if abs(x)>=1:
            out =  int(round(x, sig-int(floor(log10(abs(x))))-1))
            return f"{out:,d}" # does the right thing with commas
        else: # fractional numbers
            return str(round(x, sig-int(floor(log10(abs(x))))-1))
    except: 
        if guarantee_str:
            return guarantee_str
        return x

In [ ]:
from pylab import gca, mpl
df['year'] = df.date.dt.year
gb = df.groupby(['year','UploadKey'],as_index=False)['TotalBaseWaterVolume'].first()
gb1 = gb.groupby('year',as_index=False)['TotalBaseWaterVolume'].median()
gb1.plot('year','TotalBaseWaterVolume',ylim=(0,16500000),title='Median volume water by year')
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
gb1['rounded'] = gb1.TotalBaseWaterVolume.map(lambda x: round_sig(x,3))
gb1